In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
import torchvision
from torchvision import models
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import os

import pandas as pd
import numpy as np

from torchvision.transforms import transforms
from sklearn.metrics import roc_auc_score
from sklearn.svm import OneClassSVM

np.random.seed(252525)
torch.manual_seed(252525)

import torch
import torch.nn as nn

from meanshift import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:


device = 'cuda' if torch.cuda.is_available() else 'cpu'

net = models.vgg19(pretrained=True).features
net = net.to(device)
net = net.eval()

def flatten(X):
    return X.reshape(len(X), -1)

In [9]:
log = []
layers = [33, 34]

for run in range(3):

    for clazz in range(0, 10):

        X_, X_valid_, X_test_ = Cifar10_OneClass(train_classes=[clazz], balance=False, fold=run, transform=transforms.Compose([transforms.ToTensor()]))[0]
        X_ = X_.reshape(len(X_), 3, 32, 32)
        X_valid_ = X_valid_.reshape((len(X_valid_), 3, 32, 32))
        X_test_ = X_test_.reshape((len(X_test_), 3, 32, 32))

        X = concat_features(X_, net, layers=layers, fmap_pool=False)
        X_valid = concat_features(X_valid_, net, layers=layers, fmap_pool=False)
        X_test = concat_features(X_test_, net, layers=layers, fmap_pool=False)
        
        clf = OneClassSVM(gamma='auto').fit(flatten(X))
        s_in = -clf.score_samples(flatten(X_valid))
        s_out = -clf.score_samples(flatten(X_test))

        auc = roc_auc_score([0] * len(X_valid) + [1] * len(X_test), np.concatenate([s_in, s_out]))

        log.append(pd.DataFrame(np.asarray([run, clazz, layers[0], len(X), auc])[:, None].T, columns=["run", "class", "layers", "n_samples", "auc"])) 
    

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files alread

In [11]:
# GLOBAL layer 5/6
log_ = pd.concat(log)

for clazz in range(10):
    auc = log_[(log_["class"] == clazz)].groupby("run").mean()["auc"]
    print(f"class={Cifar10_OneClass.CLASSES[clazz]:10s}", f"\tauc: {auc.mean():.3f}", "\u00B1", f"{auc.std():.3f}")
    
log_ = pd.concat(log)

auc = log_.groupby("run").mean()["auc"]
print(f"avg.-auc: {auc.mean():.3f}", "\u00B1", f"{auc.std():.3f}")

class=plane      	auc: 0.718 ± 0.020
class=car        	auc: 0.712 ± 0.003
class=bird       	auc: 0.606 ± 0.008
class=cat        	auc: 0.643 ± 0.019
class=deer       	auc: 0.788 ± 0.008
class=dog        	auc: 0.661 ± 0.006
class=frog       	auc: 0.786 ± 0.014
class=horse      	auc: 0.704 ± 0.013
class=ship       	auc: 0.785 ± 0.003
class=truck      	auc: 0.796 ± 0.014
avg.-auc: 0.720 ± 0.009
